# Clustering Crypto

In [ ]:
import os
import pandas as pd
%matplotlib inline
import nltk
nltk.download('vader_lexicon')
from datetime import datetime

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# News Headlines Sentiment
Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:

Which coin had the highest mean positive score?
Which coin had the highest negative score?
Which coin had the highest positive score?

In [ ]:
# Read your api key environment variable
news_api_key = os.getenv("NEWS_API_KEY)

In [ ]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=news_api_key)

In [ ]:
# Creating a function to collect news articles, create sentiment analysis and dataframe with everything in it

def news_dataframe(topic):
    news = newsapi.get_everything(q='{topic}',
                #sources='bbc-news,the-verge',
                domains='bbc.co.uk,techcrunch.com',
                from_param='2019-09-23',
                to='2019-10-21',
                language='en',
                sort_by='relevancy',
                page=2)

    
    articles = []
    for article in news['articles']:
        try:
            sentiment = sid.polarity_scores(article["description"])
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]
            
            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "compound": sentiment["compound"],
                "positive": sentiment["pos"],
                "negative": sentiment["neg"],
                "neutral": sentiment["neu"],
            })
        except AttributeError:
            pass
    
    return pd.DataFrame(articles)

In [ ]:
# Run function for ethereum and bitcoin
ethereum_articles = news_dataframe('ethereum')
bitcoin_articles = news_dataframe('bitcoin')

In [ ]:
bitcoin_articles.head()

In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_articles.describe()

In [ ]:
# Describe the Ethereum Sentiment
ethereum_articles.describe()

In [ ]:
Questions:
Q: Which coin had the highest mean positive score?

A:

Q: Which coin had the highest compound score?

A:

Q. Which coin had the highest positive score?

A:

In [ ]:
Tokenizer
In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:

Lowercase each word
Remove Punctuation
Remove Stopwords

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Expand the default stop words list if necessary

In [ ]:
# Complete the tokenizer function
def tokenizer(article):
   """Tokenizes text."""
   sw = set(stopwords.words('english'))
   regex = re.compile("[^a-zA-Z ]")
   re_clean = regex.sub('', article)
   words = word_tokenize(re_clean)
   lem = [lemmatizer.lemmatize(word) for word in words]
   output = [word.lower() for word in lem if word.lower() not in sw]
   return tokens

In [ ]:
# Create a new tokens column for bitcoin
article = bitcoin_articles["description"]
tokenizer(article)

In [ ]:
# Create a new tokens column for ethereum
article = ethereum_articles["description"]
tokenizer(article)

# NGrams and Frequency Analysis
In this section you will look at the ngrams and word frequency for each coin.

Use NLTK to produce the n-grams for N = 2.
List the top 10 words for each coin.

In [ ]:
from collections import Counter
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd
lemmatizer = WordNetLemmatizer()

In [ ]:
def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return output

def bigram_counter(corpus): 
    # Combine all articles in corpus into one large string
    big_string = ' '.join(corpus)
    processed = process_text(big_string)
    bigrams = ngrams(processed, n=2)
    top_10 = dict(Counter(bigrams).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['bigram', 'count'])

def word_counter(corpus): 
    # Combine all articles in corpus into one large string
    big_string = ' '.join(corpus)
    processed = process_text(big_string)
    top_10 = dict(Counter(processed).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['word', 'count'])

In [ ]:
# Generate the Bitcoin N-grams where N=2
corpus = bitcoin_articles["description"]
bigram_counter(corpus)

In [ ]:
# Generate the Ethereum N-grams where N=2
corpus = ethereum_articles["description"]
bigram_counter(corpus)

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    # Combine all articles in corpus into one large string
    big_string = ' '.join(tokens)
    processed = process_text(big_string)
    top = dict(Counter(processed).most_common(10))
    return pd.DataFrame(list(top.items()), columns=['word', 'count'])

In [ ]:
# Get the top 10 words for Bitcoin
tokens = bitcoin_articles["description"]
token_count(tokens, N=10)

In [ ]:
# Get the top 10 words for Ethereum
tokens = ethereum_articles["description"]
token_count(tokens, N=10)

# Word Clouds
In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]
from nltk.corpus import stopwords
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
# Generate the Bitcoin word cloud
corpus = bitcoin_articles["description"]

def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return ' '.join(output)

big_string = ' '.join(corpus)
input_text = process_text(big_string)

wc = WordCloud().generate(input_text)
plt.imshow(wc)

In [ ]:
# Generate the Ethereum word cloud

corpus = ethereum_articles["description"]

big_string = ' '.join(corpus)
input_text = process_text(big_string)

wc = WordCloud().generate(input_text)
plt.imshow(wc)

# Named Entity Recognition
In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
!python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
article = ethereum_articles["description"].str.cat()

In [ ]:
# Run the NER processor on all of the text
doc = nlp(article)


# Add a title to the document

In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in doc.ents if ent.label_ == 'GPE'])

# Ethereum NER

In [ ]:
# Concatenate all of the bitcoin text together
article = bitcoin_articles["description"].str.cat()

In [ ]:
# Run the NER processor on all of the text
doc = nlp(article)

# Add a title to the document

In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in doc.ents if ent.label_ == 'GPE'])